<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/Widen00-Lab2/Lab2/Widen00_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative Methods**
**Joel Widén**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main results and conlusions.

#**About the code**


This is a report in the course DD2363 Methods in Scientific Computing. The author of this file is Joel Widén, joelwid@kth.se.

# **Set up environment**

This block is run to set up the environment.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Numbered algorithms, equations and chapter references used in this report is from the DD2363 course book Methods in Computational Science by Johan Hoffman if not stated otherwise.

**Assignment 1:** Function: Jacobi iteration for Ax=b

* *Input:* matrix $A$, vector $b$
* *Output:* vector $x$
* *Test:* convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$

**Assignment 2:** Function: Gauss-Seidel iteration for $Ax=b$

* *Input:* matrix $A$, vector $b$
* *Output:* vector $x$
* *Test:* convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$

**Assignment 3:** Function: Newton's method for scalar nonlinear equation $f(x)=0$

* *Input:* scalar function $f(x)$
* *Output:* real number $x$
* *Test:* convergence of residual $|f(x)|$, $|x-y|$ for manufactured/exact solution $y$

# **Method**

Methods and code here

In [122]:
# Assignment 1
# Jacobi iteration for Ax = b
# Jacobi iteration is based on splitting the A = A_1 + A_2 where
# A_1 = D = diag(A) and A_2 = A - D

def jacobi_iteration(A, b):
  m, n = np.shape(A)
  x = np.zeros(n)
  k = 20
  x_matrix = np.zeros((k, len(x)))
  r_vec = np.zeros(k)
  for iter in range(k):
    for i in range(0, n):
      sum = 0
      for j in range(0, n):
        if j == i:
          sum = sum
        else:
          sum = sum + A[i][j]*x[j]
      x[i] = 1/A[i][i] * (b[i] - sum)
    x_matrix[iter,:] = x
    r = np.linalg.norm(A.dot(x)-b)
    r_vec[iter] = r

  return(x_matrix, r_vec)



n = 4
m = n
# Should be diagonally dominant
# Matrix is randomized with zeroes and ones
# The maximum non diagonal sum in each row is n * 1 - 1
max_non_diag = (n * 1) - 1
# This means we must add an identity matrix with at least n * 1 - 1
# on the diagonal according to example 7.6
A = np.random.randint(2, size=(m, n)) + max_non_diag * np.eye(n)
y = np.random.randint(3, size=n)
b = A.dot(y)
D = np.diag(np.diag(A))
D_inv = np.linalg.inv(D)
Mj = np.eye(n) - np.matmul(D_inv, A)
conv_criterion = np.linalg.norm(Mj)

print("Input matrix = ", A)
print("Manufactured solution y = ", y)
print("b = ", b)
print("M = ", Mj)
print("Convergence criterion = ", conv_criterion)

x, r = jacobi_iteration(A, b)

error_matrix = x - y
error_vec = np.zeros_like(r)
i = 0
for error in error_matrix:
  error_vec[i] = np.linalg.norm(error)
  i += 1

print("Error from manufactured solution y = ",error_vec)
print("Result x = ", x[-1])
print("Result residuals r = ", r)

Input matrix =  [[3. 0. 0. 0.]
 [0. 3. 1. 1.]
 [1. 0. 3. 0.]
 [0. 1. 0. 4.]]
Manufactured solution y =  [0 0 1 1]
b =  [0. 2. 3. 4.]
M =  [[ 0.          0.          0.          0.        ]
 [ 0.          0.         -0.33333333 -0.33333333]
 [-0.33333333  0.          0.          0.        ]
 [ 0.         -0.25        0.          0.        ]]
Convergence criterion =  0.6291528696058958
Error from manufactured solution y =  [6.87184271e-01 5.72653559e-02 4.77211299e-03 3.97676083e-04
 3.31396736e-05 2.76163946e-06 2.30136622e-07 1.91780518e-08
 1.59817093e-09 1.33180958e-10 1.10984543e-11 9.24812104e-13
 7.70467405e-14 6.40859352e-15 5.29865820e-16 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Result x =  [0. 0. 1. 1.]
Result residuals r =  [1.83333333e+00 1.52777778e-01 1.27314815e-02 1.06095679e-03
 8.84130658e-05 7.36775549e-06 6.13979624e-07 5.11649683e-08
 4.26374713e-09 3.55312224e-10 2.96096481e-11 2.46735965e-12
 2.05613304e-13 1.68753900e-14 1.33226

# **Results**

Results here

# **Discussion**

Discussion here